In [1]:
import tensorflow as tf
from tensorflow.keras import Model
from genotypes import PRIMITIVES
from genotypes import Genotype
from operations import *
import numpy as np

In [2]:
tf.enable_eager_execution()

In [3]:
class MixedOp(Model):
    """Makes mixed operations object
    """

    def __init__(self, C, stride):
        """Makes ops array of all the arrays

        Args:
            C (int): the current state
        """
        super(MixedOp, self).__init__()
        self._ops = []
        for primitive in PRIMITIVES:
                op = OPS[primitive](C, stride)
                if 'pool' in primitive:
                    if('avg' in primitive):
                        op = AvgPool3x3(C, stride)
                    elif('max' in primitive):
                        op = MaxPool3x3(C, stride)
                self._ops.append(op)
        
    def call(self, x, weights):
        """Converts the discrete set of operation into conitnuous mixed operation

        Args:
            x (tensor): can be tensor or array, (e.g. image-array)
            weights (tensor): tensor or array of Softmax probability of alphas

        Returns:
            tensor: sum of product(weights, operation(x))
        """
        return sum(w * op(x) for w, op in zip(weights, self._ops))

In [4]:
class Cell(Model):

    def __init__(self, steps, multiplier, C_prev_prev, C_prev, C, reduction, reduction_prev, upsample_prev):
        super(Cell, self).__init__()
        self.reduction = reduction

        if reduction_prev:
            self.preprocess0 = FactorizedReduce(C_prev_prev, C)
        elif upsample_prev:
            self.preprocess0 = FactorizedUp(C_prev_prev, C)
        else:
            self.preprocess0 = ReLUConvBN(C_prev_prev, C, 1, 1, 'same')
        self.preprocess1 = ReLUConvBN(C_prev, C, 1, 1, 'same')
        self._steps = steps
        self._multiplier = multiplier

        self._ops = []
        self._bns = []
        for i in range(self._steps):
            for j in range(2+i):
                stride = 2 if reduction and j < 2 else 1
                op = MixedOp(C, stride)
                self._ops.append(op)

    def call(self, s0, s1, weights):
    
        s0 = self.preprocess0(s0)
        s1 = self.preprocess1(s1)

        states = [s0, s1]
        offset = 0
    
        for i in range(self._steps):
            s = sum(self._ops[offset+j](h, weights[offset+j]) for j, h in enumerate(states))
            offset += len(states)
            states.append(s)

        return tf.concat(states[-self._multiplier:], axis=-1)


In [5]:
class UpsampleCell(Model):

    def __init__(self, steps, multiplier, C_prev_prev, C_prev, C):
        super(UpsampleCell, self).__init__()
        self.preprocess0 = ReLUConvBN(C_prev_prev, C, 1, 1, 'same')
        self.preprocess1 = ReLUConvBN(C_prev, C, 1, 1, 'same')
        self._steps = steps
        self._multiplier = multiplier
        self.UpConv = layers.Conv2DTranspose(C*self._multiplier, 
                                        kernel_size=3,
                                        strides=2,
                                        padding='same')
        self.reduction = False

    def call(self, s0, s1, weights):
        s0 = self.preprocess0(s0)
        s1 = self.preprocess1(s1)

        s0 = self.UpConv(s0)
        s1 = self.UpConv(s1)

        return s0 + s1


In [32]:
class Network(Model):

    def __init__(self, C, net_layers, criterion, steps=4, multiplier=4, stem_multiplier=3):
        super(Network, self).__init__()
        self._C = C
        self._criterion = criterion
        self._steps = steps
        self._multiplier = multiplier
        self.net_layers = net_layers

        C_curr = C*stem_multiplier

        # stem operation
        self.stem_op = tf.keras.Sequential()
        self.stem_op.add(tf.keras.layers.Conv2D(
            C_curr, kernel_size=3, padding='same', use_bias=False))
        self.stem_op.add(tf.keras.layers.BatchNormalization())

        C_prev_prev, C_prev, C_curr = C_curr, C_curr, C
        self.cells = []
        self.skip_ops = []

        reduction_prev = False

        # For reduction
        for i in range(self.net_layers):
            if i % 2 == 1:
                C_curr *= 2
                reduction = True
            else:
                reduction = False
                self.skip_ops += [SkipConnection(C_curr)]
            cell = Cell(steps, multiplier, C_prev_prev, C_prev, C_curr,
                        reduction,
                        reduction_prev,
                        upsample_prev=False)
            reduction_prev = reduction
            self.cells += [cell]

            C_prev_prev, C_prev = C_prev, multiplier*C_curr

        for i in range(self.net_layers-1):
            if i % 2 == 0:
                C_curr = C_curr // 2

                cell = UpsampleCell(steps, multiplier,
                                    C_prev_prev, C_prev, C_curr)
            else:
                cell = Cell(steps, multiplier, C_prev_prev, C_prev, C_curr,
                            reduction=False,
                            reduction_prev=False,
                            upsample_prev=True)
            self.cells += [cell]
            C_prev_prev, C_prev = C_prev, multiplier*C_curr

        self.softmaxConv = tf.keras.Sequential()
        self.softmaxConv.add(tf.keras.layers.Conv2D(
            1, kernel_size=1, strides=1, padding='same'))
        self.softmaxConv.add(Softmax())

        self._initialize_alphas()

    def _initialize_alphas(self):
        k = sum(1 for i in range(self._steps) for n in range(2+i))
        num_ops = len(PRIMITIVES)

        self.alphas_normal = tf.Variable(
            1e-3*tf.random.uniform([k, num_ops]), name='alphas_normal')
        self.alphas_reduce = tf.Variable(
            1e-3*tf.random.uniform([k, num_ops]), name='alphas_reduce')
        self._arch_parameters = [
            self.alphas_normal,
            self.alphas_reduce,
        ]

    def arch_parameters(self):
        return self._arch_parameters

    def new(self):
        model_new = Network(self._C, self.net_layers, self._criterion)
        for x, y in zip(model_new.arch_parameters(), self.arch_parameters()):
            x.assign(y)
        return model_new

    def _loss(self, logits, target):
        return self._criterion(logits, tf.to_float(target))

    def call(self, inp):
        s0 = s1 = self.stem_op(inp)
        self.arr = []
        ids = []
        pos = -1

        middle = self.net_layers - 1
        for i, cell in enumerate(self.cells):
            if cell.reduction:
                weights = tf.nn.softmax(self.alphas_reduce, axis=-1)
            else:
                weights = tf.nn.softmax(self.alphas_normal, axis=-1)

            s0, s1 = s1, cell(s0, s1, weights)

            if (i < middle and i % 2 == 0):
                self.arr.append(s1)
                ids.append(i)

            if (i > middle and i % 2 == 1):
                C_curr = s1.shape[1]
                s1 = self.skip_ops[-pos-1](self.arr[pos], s1)
                pos -= 1

        return self.softmaxConv(s1)

    def genotype(self):

        def _parse(weights):
            gene = []
            n = 2
            start = 0
            for i in range(self._steps):
                end = start + n
                W = weights[start:end].copy()
                edges = sorted(range(i + 2), key=lambda x: -max(
                    W[x][k] for k in range(len(W[x])) if k != PRIMITIVES.index('none')))[:2]
                for j in edges:
                    k_best = None
                    for k in range(len(W[j])):
                        if k != PRIMITIVES.index('none'):
                            if k_best is None or W[j][k] > W[j][k_best]:
                                k_best = k
                    gene.append((PRIMITIVES[k_best], j))
                start = end
                n += 1
            return gene

            gene_normal = _parse(tf.nn.softmax(
                self.alphas_normal, axis=-1).numpy())
            gene_reduce = _parse(tf.nn.softmax(
                self.alphas_reduce, dim=-1).numpy())

            concat = range(2+self._steps-self._multiplier, self._steps+2)
            genotype = Genotype(
                normal=gene_normal, normal_concat=concat,
                reduce=gene_reduce, reduce_concat=concat
            )
            return genotype

    def get_model_thetas(self):
        specific_tensor = []
        specific_tensor_name = []
        for var in self.trainable_weights:
            if not 'alphas' in var.name:
                specific_tensor.append(var)
                specific_tensor_name.append(var.name)
        return specific_tensor

## Network Test

In [42]:
criterion = tf.losses.sigmoid_cross_entropy
network = Network(3, 3, criterion)

In [43]:
image = tf.random_uniform((1, 16, 16, 3), 0, 255)
print(network(image).shape)

(1, 16, 16, 1)


In [44]:
len(network.skip_ops)

2

In [10]:
ws2 = network.trainable_weights
for w in ws2:
    if 'aphas' in w.name:
        print(w)

<tf.Variable 'aphas_normal:0' shape=(14, 8) dtype=float32, numpy=
array([[5.2170304e-04, 4.1698923e-04, 1.9977690e-04, 8.0784172e-04,
        4.6445170e-04, 5.0997082e-04, 4.7358085e-04, 4.9465371e-04],
       [9.0265565e-04, 5.1526609e-04, 1.8710710e-04, 1.6793216e-04,
        7.4656511e-04, 7.7286118e-04, 9.8535558e-04, 6.4894737e-04],
       [1.3125516e-04, 5.0900929e-04, 2.9305305e-04, 4.3702926e-04,
        6.0322776e-04, 5.5660610e-04, 5.6299951e-04, 6.7069544e-04],
       [8.5539487e-04, 9.8293333e-04, 1.9824899e-04, 1.4187778e-04,
        6.0717302e-04, 5.0699752e-04, 5.8155990e-04, 3.2746317e-04],
       [7.9233007e-04, 4.3827895e-05, 9.3741779e-05, 7.3796028e-04,
        2.5590422e-05, 2.9862644e-05, 9.2036882e-04, 8.8963169e-04],
       [2.2841932e-04, 4.7140088e-04, 9.3648676e-04, 1.5616584e-04,
        2.5402906e-05, 9.1510528e-04, 8.1070740e-04, 2.0309497e-04],
       [3.3837880e-04, 5.2629248e-04, 9.6568436e-04, 5.9364754e-04,
        1.2094928e-04, 8.4788265e-04, 6.4325

In [10]:
network.summary()

Model: "network"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      multiple                  279       
_________________________________________________________________
cell (Cell)                  multiple                  15030     
_________________________________________________________________
cell_1 (Cell)                multiple                  57774     
_________________________________________________________________
cell_2 (Cell)                multiple                  57384     
_________________________________________________________________
cell_3 (Cell)                multiple                  225312    
_________________________________________________________________
cell_4 (Cell)                multiple                  224064    
_________________________________________________________________
cell_5 (Cell)                multiple                  8895

In [13]:
network.get_weights()

[array([[[[-0.09391835, -0.06674695, -0.14879438, -0.07497576,
           -0.11925114, -0.11194815, -0.1966327 ,  0.14210163,
           -0.05619664],
          [-0.23255788, -0.16882223, -0.13532594, -0.19584203,
           -0.05881704,  0.03954403,  0.22381152, -0.12375265,
            0.2108268 ],
          [ 0.09032463,  0.1756448 , -0.04163896, -0.0689197 ,
            0.11407886, -0.17627408,  0.15092088,  0.17241   ,
           -0.10629778]],
 
         [[-0.18053626, -0.03851373,  0.13697092, -0.14150792,
            0.16338395,  0.17333992, -0.1460408 , -0.05304798,
            0.16342424],
          [-0.23459515,  0.07835914,  0.06850542,  0.2082613 ,
           -0.1489405 ,  0.19954674, -0.08756413,  0.15136714,
            0.1127082 ],
          [-0.17484316, -0.02526271,  0.06235902,  0.19912536,
           -0.13127118, -0.2224187 ,  0.08863218,  0.09770931,
           -0.16266626]],
 
         [[ 0.09735771,  0.12866516, -0.16005288,  0.10500304,
           -0.14848223,  

# Testing Here!

In [14]:
import numpy as np

## Mixed Op

In [15]:
mix_op = MixedOp(3, stride=1)

In [16]:
weights = np.random.rand(8).astype(np.float32)
features = np.random.rand(2, 16, 16, 3).astype(np.float32)

init = tf.global_variables_initializer()

out = mix_op(features, weights)

## Cell

In [17]:
cell = Cell(steps=4,
            multiplier=4,
            C_prev_prev=3,
            C_prev=3,
            C=3,
            reduction=False,
            reduction_prev=False,
            upsample_prev=False)

In [18]:
weights = np.random.rand(20,8).astype(np.float32)
s0 = np.random.rand(2, 16, 16, 3).astype(np.float32)
s1 = np.random.rand(2, 16, 16, 3).astype(np.float32)

init = tf.global_variables_initializer()

cell_out = cell(s0, s1, weights)

In [19]:
cell_out

<tf.Tensor: id=108697, shape=(2, 16, 16, 12), dtype=float32, numpy=
array([[[[ 8.70200634e-01,  1.02092814e+00, -1.36692643e+00, ...,
           5.89161825e+00,  2.37572327e+01, -1.39936895e+01],
         [ 3.11145425e-01,  1.04163265e+00, -1.10464966e+00, ...,
           4.59814930e+00,  2.58104477e+01, -1.01848831e+01],
         [ 2.89471537e-01,  1.86336339e+00, -6.24090374e-01, ...,
           5.66164017e+00,  2.97692146e+01, -8.46916962e+00],
         ...,
         [ 6.04736269e-01,  1.70712388e+00, -1.40251267e+00, ...,
           1.15938206e+01,  2.23939438e+01, -1.52867374e+01],
         [ 6.79464400e-01,  1.69136441e+00, -9.94855702e-01, ...,
           1.55685863e+01,  2.95255222e+01, -1.26513710e+01],
         [ 8.46070766e-01,  1.81877959e+00, -5.36994398e-01, ...,
           1.48124027e+01,  3.08296661e+01, -1.38808155e+01]],

        [[ 5.40972471e-01,  1.93342638e+00, -1.12626433e+00, ...,
           7.28219080e+00,  2.70997219e+01, -1.19970713e+01],
         [ 8.1942692

## Upsample Cell

In [ ]:
up_cell = UpsampleCell(4, 4, 3, 3, 3)

In [ ]:
s0 = np.random.rand(2, 16, 16, 3).astype(np.float32)
s1 = np.random.rand(2, 16, 16, 3).astype(np.float32)

init = tf.global_variables_initializer()

up_cell_out = up_cell(s0, s1)

## Extra Test

In [ ]:
s0 = np.random.rand(2, 16, 16, 6).astype(np.float32)
conv = tf.keras.Sequential()
conv.add(ReLUConvBN(6, 6, 1, 1, 'same'))
conv.add(FactorizedReduce(6,6))
conv(s0)

In [19]:
var = tf.Variable([[0.9], [0.5]])
var2 = tf.Variable([[0.9], [0.5]])

In [22]:
Zero(2).name

'zero_280'